In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [10]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.float)

print(bits_info)

tensor([[1., 0., 1., 0.],
        [1., 1., 1., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 0.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 1., 1.],
        [1., 1., 0., 1.],
        [1., 1., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0.,

Hamming(7,4) Encoder

In [11]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.float)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [20]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1., 0., 1., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1., 0., 1.],
        [1.,

BPSK Modulator + Noise

In [44]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        super(bpsk_modulator, self).__init__()
        
    def forward(self, data, symbol_rate, carrier_freq, snr_dB):
    # data = encoded_codeword  # Binary data
    # symbol_rate = 15  # Symbol rate in Hz
    # carrier_freq = 100  # Carrier frequency in Hz
    # snr_dB = 15  # Signal-to-noise ratio in dB
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
            # Time vector
            time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
        
            # Generate carrier signal
            carrier = torch.cos(2 * torch.pi * carrier_freq * time)
        
            # Modulate the signal
            modulated_signal = bits * carrier
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(-snr_dB / 10))
            noise = torch.sqrt(noise_power) * torch.randn(len(modulated_signal))
            noised_signal = modulated_signal + noise
            data[i] = noised_signal
    
       
        return time, data

In [45]:
data = encoded_codeword  # Binary data
symbol_rate = 15  # Symbol rate in Hz
carrier_freq = 100  # Carrier frequency in Hz
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
time, modulated_noise_signal = modulator(data, symbol_rate, carrier_freq, snr_dB)
print(modulated_noise_signal)

tensor([[-6.8888e+01,  7.8937e-01,  1.0538e+00, -9.8632e+02,  1.4646e+00,
         -5.8749e-01,  1.5509e+02],
        [ 3.7947e+00,  7.2500e-01,  4.8531e-01,  8.3779e+00,  6.4908e-01,
          1.3330e+00, -6.1804e+00],
        [-6.0789e+01, -8.9506e-01,  8.5928e-01, -6.5929e+01, -1.2373e-01,
          1.0041e+00, -2.1718e+00],
        [-6.3072e+01,  4.8879e-01,  6.9116e-02,  1.0759e+01,  7.2078e-01,
          1.3614e+00,  7.6758e+00],
        [-7.3809e+01, -9.0896e-01,  1.7412e+00, -6.5468e+01,  3.2583e-01,
          1.1145e+00, -3.2571e+00],
        [-7.8560e+00,  9.9764e-01, -4.6514e-01, -2.3710e+00,  1.4306e+00,
         -2.1361e-01, -7.0070e+01],
        [ 2.2480e+00,  6.6214e-02,  1.0461e+00,  6.7194e+00,  4.1412e-01,
          1.6384e+00, -7.5380e+01],
        [ 6.5163e+00,  7.4753e-01,  1.0006e+00,  6.5015e+00,  1.2396e+00,
          5.7582e-01,  3.0482e+00],
        [-7.1261e+01, -1.9326e-01,  1.0190e+00,  3.8831e+00,  1.8973e-01,
          1.5131e+00, -6.8261e+00],
        [ 

LLR Log-likelihood

In [147]:
# import numpy as np
# 
# def ldpc_decode(received_codeword, max_iterations=50):
#     # LDPC Parity-check matrix (7,4) example
#     H = np.array([
#         [1, 0, 1, 1, 0, 0, 0],
#         [0, 1, 0, 0, 1, 0, 1],
#         [0, 0, 0, 1, 0, 1, 0]
#     ])
# 
#     # LDPC Generator matrix (7,4) example
#     G = np.array([
#         [1, 0, 0, 0],
#         [0, 1, 0, 0],
#         [0, 0, 1, 0],
#         [0, 0, 0, 1],
#         [0, 1, 1, 1],
#         [1, 0, 1, 1],
#         [1, 1, 0, 1]
#     ])
# 
#     # Initialize the received codeword and the decoded message
#     y = np.array(list(map(int, received_codeword)))
#     x_hat = np.zeros(G.shape[1])
# 
#     for iteration in range(max_iterations):
#         # Syndrome check
#         syndrome = np.mod(np.dot(H, y), 2)
# 
#         if np.all(syndrome == 0):
#             # If the syndrome is all zeros, the decoding is successful
#             break
# 
#         # Compute the log-likelihood ratios (LLRs) using the sum-product algorithm
#         llrs = np.zeros(G.shape[1])
#         for j in range(G.shape[1]):
#             connected_checks = np.nonzero(H[:, j])[0]
#             llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))
# 
#         # Make hard decisions based on LLRs
#         x_hat = np.where(llrs > 0, 1, 0)
# 
#         # Update the received codeword
#         y = np.mod(np.dot(G, x_hat) + y, 2)
# 
#     decoded_message = ''.join(map(str, x_hat.astype(int)))
#     return decoded_message
# 
# def main():
#     # Example usage
#     received_codeword = "1101001"  # Received codeword with possible errors
#     decoded_message = ldpc_decode(received_codeword)
# 
#     print(f"Received codeword: {received_codeword}")
#     print(f"Decoded message: {decoded_message}")
# 
# if __name__ == "__main__":
#     main()
